In [2]:
import pandas as pd #đọc dữ liệu
import numpy as np #xử lý dữ liệu
import matplotlib.pyplot as plt #vẽ biểu đồ

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Concatenate, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, concatenate

from sklearn.model_selection import KFold

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df=pd.read_csv('URLdatasetX2_1.csv')
# df=df.drop(columns=["ID"])

print(df)

      Unnamed: 0                                                url  \
0              0              http://www.crestonwood.com/router.php   
1              1         http://vamoaestudiarmedicina.blogspot.com/   
2              2  https://www.astrologyonline.eu/Astro_MemoNew/P...   
3              3        https://www.lifewire.com/tcp-port-21-818146   
4              4  https://technofizi.net/top-best-mp3-downloader...   
...          ...                                                ...   
2797        2797            http://apuwa.com/blog/profile/name.html   
2798        2798                         https://www.dissernet.org/   
2799        2799          http://www.heinzreber.net/homeflash1.html   
2800        2800  https://adnanboz.wordpress.com/2012/01/06/how-...   
2801        2801  https://www.facebook.com/Interactive-Televisio...   

            type  
0     legitimate  
1     legitimate  
2     legitimate  
3     legitimate  
4     legitimate  
...          ...  
2797    phishi

In [11]:
df['type'] = df['type'].map({'legitimate': 0, 'phishing': 1})


In [12]:
# df['type'] = df['type'].replace({'phishing': 1, 'legitimate': 0})
datas=df["url"]
label=df["type"]

In [18]:
print(len(datas))
print(label[1])
print(df.head(5))

2802
0
   Unnamed: 0                                                url  type
0           0              http://www.crestonwood.com/router.php     0
1           1         http://vamoaestudiarmedicina.blogspot.com/     0
2           2  https://www.astrologyonline.eu/Astro_MemoNew/P...     0
3           3        https://www.lifewire.com/tcp-port-21-818146     0
4           4  https://technofizi.net/top-best-mp3-downloader...     0


In [ ]:

X_train=datas[:2441]
y_train=label[:2441]
X_test=datas[2441:]
y_test=label[2441:]
print(X_train)
print(y_test)

0                   http://www.crestonwood.com/router.php
1              http://vamoaestudiarmedicina.blogspot.com/
2       https://www.astrologyonline.eu/Astro_MemoNew/P...
3             https://www.lifewire.com/tcp-port-21-818146
4       https://technofizi.net/top-best-mp3-downloader...
                              ...                        
2436     https://de.gethuman.com/customer-service/Netflix
2437                    http://ngampusonline.blogspot.com
2438               http://vtchekhov.com/sa//?sec=AnnaLisa
2439                               https://beta.znipe.tv/
2440      http://www.jkang.com/CIS110/ch07_bis/sld015.htm
Name: url, Length: 2441, dtype: object
2441    0
2442    0
2443    0
2444    0
2445    0
       ..
2797    1
2798    0
2799    1
2800    0
2801    0
Name: type, Length: 361, dtype: int64


In [ ]:
print(X_train)
print(y_test)

0                   http://www.crestonwood.com/router.php
1              http://vamoaestudiarmedicina.blogspot.com/
2       https://www.astrologyonline.eu/Astro_MemoNew/P...
3             https://www.lifewire.com/tcp-port-21-818146
4       https://technofizi.net/top-best-mp3-downloader...
                              ...                        
2436     https://de.gethuman.com/customer-service/Netflix
2437                    http://ngampusonline.blogspot.com
2438               http://vtchekhov.com/sa//?sec=AnnaLisa
2439                               https://beta.znipe.tv/
2440      http://www.jkang.com/CIS110/ch07_bis/sld015.htm
Name: url, Length: 2441, dtype: object
2441    0
2442    0
2443    0
2444    0
2445    0
       ..
2797    1
2798    0
2799    1
2800    0
2801    0
Name: type, Length: 361, dtype: int64


In [ ]:
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [ ]:
y_test[0]

array([1., 0.], dtype=float32)

In [ ]:
X_train.shape

(2441,)

In [7]:
tokenizer=Tokenizer(char_level=True)

In [8]:
def change_data_char(X_train):
  max_sequence_length=max(len(url) for url in X_train)
  tokenizer.fit_on_texts(X_train)
  encoded_sequences=tokenizer.texts_to_sequences(X_train)
  padded=pad_sequences(encoded_sequences,padding="post",maxlen=200)
  return padded
  # padded la ma tran cac ky tu duoc ma hoa trong ca link

In [14]:
def change_data_word_char(X_train):
  list_word_in_url=[]
  # max_word_unit=0
  # max_word=0
  for url in X_train:
    tokens=re.findall(r'\w+|\S',url)
    # max_word_unit=max(max_word_unit,max(len(a) for a in tokens))
    # max_word=max(max_word,len(tokens))
    token=' '.join(tokens)
    list_word_in_url.append(token)
  tokenizer_word = Tokenizer(filters='')
  tokenizer_word.fit_on_texts(list_word_in_url)
  sequences_word = tokenizer_word.texts_to_sequences(list_word_in_url)
  padded_1=pad_sequences(sequences_word,padding="post",truncating="post",maxlen=200)

  pad_token_sum=[]
  for url in list_word_in_url:
    token_char_unit=tokenizer.texts_to_sequences(url)
    # pad_token_char_unit=pad_sequences(token_char_unit,padding="post",maxlen=max_word_unit)
    pad_token_char_unit=pad_sequences(token_char_unit,padding="post",maxlen=500)
    pad_token_sum.append(pad_token_char_unit)

  # padded_1 la ma tran ma hoa tu trong cac link
  # pad_token_sum la ma tran ma hoa ky tu theo cac tu trong link
  pad_pad_token_sum=pad_sequences(pad_token_sum,padding="post",maxlen=200)
  # pad_pad_token_sum=pad_sequences(pad_token_sum,padding="post",maxlen=max_word)
  pad_pad_token_sum=pad_pad_token_sum

  URL_cw=np.sum(pad_pad_token_sum,axis=2)
  URL_w_cw=URL_cw+padded_1
  return URL_w_cw

In [15]:








#CNN for URL_w_cw and padded


y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
padded=change_data_char(X_train)
input_shape=padded.shape[1]
# Khởi tạo mô hình cho ma trận padded 472x5000
model_1 = Sequential()
model_1.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(input_shape,1)))
model_1.add(MaxPooling1D(pool_size=2,strides=2))
model_1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model_1.add(MaxPooling1D(pool_size=2,strides=2))
model_1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model_1.add(MaxPooling1D(pool_size=2,strides=2))
# model_1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# model_1.add(MaxPooling1D(pool_size=2,strides=2))
# model_1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# model_1.add(MaxPooling1D(pool_size=2,strides=2))
# model_1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# model_1.add(MaxPooling1D(pool_size=2,strides=2))
model_1.add(Flatten())
out_model_1= Dense(128, activation='relu')(model_1.output)
URL_w_cw=change_data_word_char(X_train)
input_shape_1=URL_w_cw.shape[1]
# Khởi tạo mô hình cho ma trận URL_w_cw
model_2 = Sequential()
model_2.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(input_shape_1,1)))
model_2.add(MaxPooling1D(pool_size=2,strides=2))
model_2.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model_2.add(MaxPooling1D(pool_size=2,strides=2))
model_2.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model_2.add(MaxPooling1D(pool_size=2,strides=2))
# model_2.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# model_2.add(MaxPooling1D(pool_size=2,strides=2))
# model_2.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# model_2.add(MaxPooling1D(pool_size=2,strides=2))
model_2.add(Flatten())
out_model_2= Dense(128, activation='relu')(model_2.output)
# Kết hợp đầu ra của hai mô hình trước vào một fully connected layer
merged = concatenate([out_model_1, out_model_2])
merged = Dense(128, activation='relu')(merged)
merged = Dense(64, activation='relu')(merged)
# Lớp output với 2 nơ-ron và hàm kích hoạt softmax
output_layer = Dense(2, activation='softmax')(merged)

# Tạo model từ Input và output
combined_model = Model(inputs=[model_1.input, model_2.input], outputs=output_layer)

# Compile model
combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# In cấu trúc của mô hình
combined_model.summary()

padded=change_data_char(X_train)
URL_w_cw=change_data_word_char(X_train)
combined_model.fit([padded, URL_w_cw], y_train, epochs=5, batch_size=32, validation_split=0.2)

predict_padded=change_data_char(X_test)
predict_URL_w_cw=change_data_word_char(X_test)
predictions = combined_model.predict([predict_padded,predict_URL_w_cw])
print(predictions[0:5])
# print(predict_padded[0])

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
predicted_url_single = np.argmax(predictions, axis=1)
actual_url_single = np.argmax(y_test, axis=1)
accuracy = accuracy_score(actual_url_single, predicted_url_single)
precision = precision_score(actual_url_single, predicted_url_single, average='macro')
recall = recall_score(actual_url_single, predicted_url_single, average='macro')
f1 = f1_score(actual_url_single, predicted_url_single, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

NameError: name 're' is not defined

In [ ]:
print(len(datas))

2802


In [21]:
from sklearn.model_selection import KFold
import re

n_loops = 1
n_folds = 5
base_dir = 'result'
n_samples = len(datas)
results_df = pd.DataFrame(columns=['fold', 'Train Acc', 'Test Acc'])
for i in range(n_loops):
  cv = KFold(n_splits=n_folds, shuffle=True, random_state = i)
  for fold, (train_idx, test_idx) in enumerate(cv.split(datas)):
    path_dir = base_dir +'/' +'URLdatasetX2_1'+ '_run_'+str(i)+'_'+ 'fold_'+str(fold)+'_'
    print('Run: ', i, ', fold: ', fold)
    X_train = datas[train_idx]
    X_test = datas[test_idx]
    y_train = label[train_idx]
    y_test = label[test_idx]
    y_train=to_categorical(y_train)
    y_test=to_categorical(y_test)
    padded=change_data_char(X_train)
    input_shape=padded.shape[1]
    # Khởi tạo mô hình cho ma trận padded 472x5000
    model_1 = Sequential()
    model_1.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(input_shape,1)))
    model_1.add(MaxPooling1D(pool_size=2,strides=2))
    model_1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model_1.add(MaxPooling1D(pool_size=2,strides=2))
    model_1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model_1.add(MaxPooling1D(pool_size=2,strides=2))
    # model_1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    # model_1.add(MaxPooling1D(pool_size=2,strides=2))
    # model_1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    # model_1.add(MaxPooling1D(pool_size=2,strides=2))
    # model_1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    # model_1.add(MaxPooling1D(pool_size=2,strides=2))
    model_1.add(Flatten())
    out_model_1= Dense(128, activation='relu')(model_1.output)
    URL_w_cw=change_data_word_char(X_train)
    input_shape_1=URL_w_cw.shape[1]
    # Khởi tạo mô hình cho ma trận URL_w_cw
    model_2 = Sequential()
    model_2.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(input_shape_1,1)))
    model_2.add(MaxPooling1D(pool_size=2,strides=2))
    model_2.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model_2.add(MaxPooling1D(pool_size=2,strides=2))
    model_2.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model_2.add(MaxPooling1D(pool_size=2,strides=2))
    # model_2.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    # model_2.add(MaxPooling1D(pool_size=2,strides=2))
    # model_2.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    # model_2.add(MaxPooling1D(pool_size=2,strides=2))
    model_2.add(Flatten())
    out_model_2= Dense(128, activation='relu')(model_2.output)
    # Kết hợp đầu ra của hai mô hình trước vào một fully connected layer
    merged = concatenate([out_model_1, out_model_2])
    merged = Dense(128, activation='relu')(merged)
    merged = Dense(64, activation='relu')(merged)
    # Lớp output với 2 nơ-ron và hàm kích hoạt softmax
    output_layer = Dense(2, activation='softmax')(merged)

    # Tạo model từ Input và output
    combined_model = Model(inputs=[model_1.input, model_2.input], outputs=output_layer)

    # Compile model
    combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # In cấu trúc của mô hình
    #combined_model.summary()

    padded=change_data_char(X_train)
    URL_w_cw=change_data_word_char(X_train)
    combined_model.fit([padded, URL_w_cw], y_train, epochs=5, batch_size=32, validation_split=0.2)

    predict_padded=change_data_char(X_test)
    predict_URL_w_cw=change_data_word_char(X_test)
    predictions = combined_model.predict([predict_padded,predict_URL_w_cw])
    y_score = predictions[:,1]

    print(predictions[0:5])
    # print(predict_padded[0])

    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    predicted_url_single = np.argmax(predictions, axis=1)
    actual_url_single = np.argmax(y_test, axis=1)
    accuracy = accuracy_score(actual_url_single, predicted_url_single)
    precision = precision_score(actual_url_single, predicted_url_single, average='macro')
    recall = recall_score(actual_url_single, predicted_url_single, average='macro')
    f1 = f1_score(actual_url_single, predicted_url_single, average='macro')

    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)

    df = pd.DataFrame({'Actual': actual_url_single, 'Predicted': predicted_url_single, 'y_score':y_score})
    df.to_csv(path_dir + "URLNet_labels.csv", index=False)

Run:  0 , fold:  0
Epoch 1/5
56/56 [==============================] - 5s 39ms/step - loss: 1.7947 - accuracy: 0.7188 - val_loss: 1.3091 - val_accuracy: 0.7506
Epoch 2/5
56/56 [==============================] - 1s 22ms/step - loss: 0.5689 - accuracy: 0.7879 - val_loss: 1.1582 - val_accuracy: 0.6971
Epoch 3/5
56/56 [==============================] - 1s 22ms/step - loss: 0.4797 - accuracy: 0.7885 - val_loss: 0.9471 - val_accuracy: 0.7550
Epoch 4/5
56/56 [==============================] - 1s 21ms/step - loss: 0.4281 - accuracy: 0.8041 - val_loss: 0.8316 - val_accuracy: 0.7862
Epoch 5/5
18/18 [==============================] - 0s 6ms/step
[[0.42700225 0.57299775]
 [0.36408722 0.6359128 ]
 [0.00933285 0.99066716]
 [0.5604344  0.4395656 ]
 [0.5444848  0.45551518]]
Accuracy: 0.7575757575757576
Precision: 0.7153258501337598
Recall: 0.7415761626287942
F1-score: 0.7234770506538317
Run:  0 , fold:  1
Epoch 1/5
56/56 [==============================] - 4s 28ms/step - loss: 1.3754 - accuracy: 0.7321 

In [ ]:
# Huấn luyện mô hình
print(y_train.shape)
padded=change_data_char(X_train)
URL_w_cw=change_data_word_char(X_train)
combined_model.fit([padded, URL_w_cw], y_train, epochs=300, batch_size=32, validation_split=0.2)

In [ ]:
predict_padded=change_data_char(X_test)
predict_URL_w_cw=change_data_word_char(X_test)
predictions = combined_model.predict([predict_padded,predict_URL_w_cw])
print(predictions[0:5])
# print(predict_padded[0])

12/12 [==============================] - 0s 3ms/step
[[1.0000000e+00 1.0526151e-14]
 [1.0000000e+00 1.3614529e-10]
 [1.2462253e-01 8.7537754e-01]
 [9.8300320e-01 1.6996834e-02]
 [1.0000000e+00 7.9354383e-15]]


In [ ]:
y_test[0:5]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
#Chuyển đổi mảng dự đoán thành các nhãn dơn lẻ
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
predicted_url_single = np.argmax(predictions, axis=1)
actual_url_single = np.argmax(y_test, axis=1)
accuracy = accuracy_score(actual_url_single, predicted_url_single)
precision = precision_score(actual_url_single, predicted_url_single, average='macro')
recall = recall_score(actual_url_single, predicted_url_single, average='macro')
f1 = f1_score(actual_url_single, predicted_url_single, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.8310249307479224
Precision: 0.7927000140114895
Recall: 0.8126309488177192
F1-score: 0.8011127067132702


In [ ]:
print(predicted_url_single)
print(actual_url_single)

[0 0 1 1 0 1 0 0 1 1 0 0 0 0 1 0 1 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1
 1 1 0 0 1 1 0 0 1 0 1 1 1 0 1 0 1 1 1 1 0 0 1 0 0 1 0 1 1 0 0 0 0 1 0 0 0
 0 1 1 0 0 0 0 0 0 1 0 1 0 1 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0
 0 0 0 0 0 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 1 0 1
 1 0 1 0 0 0 1 0 0 0 1 1 0 0 1 0 1 1 0 0 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0
 0 0 0 0 1 0 0 0 1 0 1 0 0 1 1 0 0 1 0 1 1 0 1 0 0 1 1 0 1 0 0 0 1 0 0 0 1
 1 1 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 0 1 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0
 1 1 1 0 1 1 1 0 1 0 0 1 1 0 0 0 1 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 1 0

In [ ]:
# import csv
# import re
# import requests
# from requests.exceptions import InvalidURL

# def check_url(url):
#     # Kiểm tra tính hợp lệ của URL bằng biểu thức chính quy
#   if re.match(r'^https?://\S+', url):
#       try:
#           response = requests.head(url,timeout=3)
#           return response.status_code==200
#       except Exception as e:
#           return False
#   else:
#       return False

# # Đọc danh sách các URL từ tệp CSV
# url_listt = pd.read_csv('/content/drive/MyDrive/PhisingURL/malicious_phish_LanLon.csv')
# url_list=url_listt["url"][130000:140000]

# # Thêm scheme vào mỗi URL
# url_list_with_scheme = ['http://' + url if not url.startswith(('http://', 'https://')) else url for url in url_list]

# # Kiểm tra sự hoạt động của mỗi URL
# count=130000
# results = []
# for url in url_list_with_scheme:
#     if check_url(url):
#       results.append([url,url_listt["type"][count]])
#       # results[url]=check_url(url)
#       print(f"{count}:{url}: {'hoạt động' if results else 'không hoạt động'}")
#     count+=1
# # results=pd.DataFrame.from_dict(results)
# results=pd.DataFrame(results, columns=['url', 'type'])
# results.to_csv('/content/drive/MyDrive/PhisingURL/malicious_phish_LanLon_14.csv')
# # # In kết quả
# # for url, result in results.items():
# #     print(f"{url}: {'hoạt động' if result else 'không hoạt động'}")


In [ ]:
# import csv
# import re
# import requests
# from requests.exceptions import InvalidURL

# def check_url(url):
#     # Kiểm tra tính hợp lệ của URL bằng biểu thức chính quy
#       try:
#           response = requests.head(url,timeout=3)
#           return response.status_code==200
#       except Exception as e:
#           return False

# # Đọc danh sách các URL từ tệp CSV
# url_listt = pd.read_csv('/content/drive/MyDrive/PhisingURL/phished-urls_CoMien.csv')
# url_list=url_listt["Url"]
# url_headers=url_listt.columns.tolist()
# # Thêm scheme vào mỗi URL
# # url_list_with_scheme = ['http://' + url if not url.startswith(('http://', 'https://')) else url for url in url_list]

# # Kiểm tra sự hoạt động của mỗi URL
# count=0
# results = []
# for url in url_list:
#     if check_url(url):
#       results.append(url_listt.iloc[count].tolist())
#       # results[url]=check_url(url)
#       print(f"{count}:{url}: {'hoạt động' if results else 'không hoạt động'}")
#     count+=1
# # results=pd.DataFrame.from_dict(results)
# results=pd.DataFrame(results, columns=url_headers)
# results.to_csv('/content/drive/MyDrive/PhisingURL/phished-urls_CoMien_1.csv')
# # # In kết quả
# # for url, result in results.items():
# #     print(f"{url}: {'hoạt động' if result else 'không hoạt động'}")


In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # Đọc dữ liệu từ file CSV vào DataFrame
# df = pd.read_csv('/content/drive/MyDrive/PhisingURL/new_data_urls_KoMien.csv')

# # Lọc các mẫu có nhãn là bad
# df_label_0 = df[df['status'] == 1]

# # Lấy mẫu ngẫu nhiên 5000 mẫu từ nhóm nhãn 0
# sample_label_0 = df_label_0.sample(n=2000, random_state=42)

# # Lọc các mẫu có nhãn là 1
# df_label_1 = df[df['status'] == 0]

# # Lấy mẫu ngẫu nhiên 5000 mẫu từ nhóm nhãn 1
# sample_label_1 = df_label_1.sample(n=8000, random_state=42)

# # Kết hợp hai mẫu lại với nhau
# final_sample = pd.concat([sample_label_0, sample_label_1])

# # Ghi ra file CSV mới
# final_sample.to_csv('/content/drive/MyDrive/PhisingURL/data1_new_data_urls_10k.csv', index=False)
# # Chia dữ liệu thành tập train và test
# train_data, test_data = train_test_split(final_sample, test_size=0.2, random_state=42)
# # Ghi ra file CSV mới
# train_data.to_csv('/content/drive/MyDrive/PhisingURL/data1_10k_new_data_urls_train.csv', index=False)
# # Ghi ra file CSV mới
# test_data.to_csv('/content/drive/MyDrive/PhisingURL/data1_10k_new_data_urls_test.csv', index=False)


In [ ]:
import shutil

# Nén thư mục /content/results thành tệp results.zip
shutil.make_archive('/content/result', 'zip', '/content/result')

# Sử dụng hàm download của Google Colab để tải tệp .zip về máy
from google.colab import files
files.download('/content/result.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>